# Ingest data with Redshift
This notebook demonstrates how to set up database with Redshift and query data with it. 

Amazon Redshift is a fully managed data warehouse which allows you to run complex analytic queries against petabytes of structured data. Your queries are distributed and parallelized across multiple physical resources, and you can easily scale your Amazon Redshift environment up and down depending on your business needs.

**Difference from the [existing notebook](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/working_with_redshift_data/working_with_redshift_data.ipynb):**
* use Secret Manager instead of a json file for credentials for better cred management
* show how to attach required policies in the notebook
* set up redshift cluster from scratch in the notebook
* using `sqlalchemy` for sql engine and querying (instead of Psycopg -- PostgreSQL database adapter)

## When should you use Redshift?

While Athena is mostly used to run ad-hoc queries on Amazon S3 data lake, Redshift is usually recommended for large structured data sets, or traditional relational database; it does well with performing aggregations, complex joins, and inner queries. You would need to set up and load the cluster before using it; and you need to load data into created tables. 

## Set up Redshift
First we are going to make sure we have policy attached to our role to access Redshift. You can do this through IAM client as below, or through the AWS console.

In [10]:
import io
import boto3
import sagemaker
import json
from sagemaker import get_execution_role
import os
import sys

# Get region 
session = boto3.session.Session()
region_name = session.region_name

# Get SageMaker session & default S3 bucket
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket() #replace with your own bucket name if you have one
role = sagemaker.get_execution_role()

In [11]:
from botocore.exceptions import ClientError
from botocore.config import Config

iam = boto3.client('iam')
sts = boto3.client('sts')
redshift = boto3.client('redshift')

In [12]:
role_name = role.split('/')[-1]

print('Role name: {}'.format(role_name))

Role name: AmazonSageMaker-ExecutionRole-20201006T125078


### Create Redshift Role
The policy enables Redshift to assume the role. The services can then perform any tasks granted by the permissions policy assigned to the role (which we will attach to it later). 

In [13]:
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "redshift.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

In [14]:
# Create Role
iam_redshift_role_name = 'Tabular_Redshift'
try:
    iam_role_redshift = iam.create_role(
        RoleName=iam_redshift_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='Tabular data Redshift Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

Role already exists


In [15]:
#get role arn
role = iam.get_role(RoleName='Tabular_Redshift')
iam_role_redshift_arn = role['Role']['Arn']
print(iam_role_redshift_arn)

arn:aws:iam::060356833389:role/Tabular_Redshift


### Create Policy Document
We will create policies we used to access S3 and Athena. The two policies we will create here are: 
* S3FullAccess: `arn:aws:iam::aws:policy/AmazonS3FullAccess`
* AthenaFullAccess: `arn:aws:iam::aws:policy/AmazonAthenaFullAccess`

You can check the policy document in the IAM console and copy the policy file here.

In [16]:
#s3FullAccess
my_redshift_to_s3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": "*"
        }
    ]
}

In [17]:
#Athena Full Access
my_redshift_to_athena = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "athena:*"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload",
                "s3:CreateBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::aws-athena-query-results-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::athena-examples*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:ListAllMyBuckets"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "sns:ListTopics",
                "sns:GetTopicAttributes"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricAlarm",
                "cloudwatch:DescribeAlarms",
                "cloudwatch:DeleteAlarms"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "lakeformation:GetDataAccess"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

In [18]:
try:
    policy_redshift_s3 = iam.create_policy(
      PolicyName='Tabular_RedshiftPolicyToS3',
      PolicyDocument=json.dumps(my_redshift_to_s3)
    )
    print ('Policy created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print ("Policy already exists")
    else:
        print ("Unexpected error: %s" % e)

account_id = sts.get_caller_identity()['Account']
policy_redshift_s3_arn = f'arn:aws:iam::{account_id}:policy/Tabular_RedshiftPolicyToS3'

Policy already exists


In [19]:
try:
    policy_redshift_athena = iam.create_policy(
      PolicyName='Tabular_RedshiftPolicyToAthena',
      PolicyDocument=json.dumps(my_redshift_to_athena)
    )
    print ('Policy created.')
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print ("Policy already exists")
    else:
        print ("Unexpected error: %s" % e)
        
account_id = sts.get_caller_identity()['Account']
policy_redshift_athena_arn = f'arn:aws:iam::{account_id}:policy/Tabular_RedshiftPolicyToAthena'

Policy already exists


### Attach Policy to Role

In [20]:
# Attach RedshiftPolicyToAthena policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_athena_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [21]:
# Attach RedshiftPolicyToS3 policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_s3_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

### Making Sure your Role to run this Notebook has the following policy attached:

* `SecretsManagerReadWrite`: we will use this service to store and retrive our Redshift Credentials.
* `AmazonRedshiftFullAccess`: we will use this role to create a Redshift cluster from the notebook.

In [22]:
#making sure you have secret manager policy attached to role
try:
    policy='SecretsManagerReadWrite'
    response = iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/{}'.format(policy),
        RoleName=role_name
    )
    print("Policy %s has been succesfully attached to role: %s" % (policy, role_name))
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached.")
    else:
        print("Unexpected error: %s " % e)

Policy SecretsManagerReadWrite has been succesfully attached to role: AmazonSageMaker-ExecutionRole-20201006T125078


In [23]:
#making sure you have RedshiftFullAccess policy attached to role
from botocore.exceptions import ClientError
try:
    policy='AmazonRedshiftFullAccess'
    response = iam.attach_role_policy(
        PolicyArn='arn:aws:iam::aws:policy/{}'.format(policy),
        RoleName=role_name
    )
    print("Policy %s has been succesfully attached to role: %s" % (policy, role_name))
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. ")
    else:
        print("Unexpected error: %s " % e)

Policy AmazonRedshiftFullAccess has been succesfully attached to role: AmazonSageMaker-ExecutionRole-20201006T125078


# Optional: Create Redshift Cluster

Most of the times we have a Redshift cluster already up and running and we want to connect to the cluster in-use, but if you want to create a new cluster, you can follow the steps below to create one.
*Note that only some Instance Types support Redshift Query Editor, so be careful when you specify the Redshift Cluster Nodes.*(https://docs.aws.amazon.com/redshift/latest/mgmt/query-editor.html).

### Get Security Group ID


In [24]:
import sagemaker
sm = boto3.client('sagemaker')

notebook_instance_name = sm.list_notebook_instances()['NotebookInstances'][0]['NotebookInstanceName']

In [25]:
notebook_instance = sm.describe_notebook_instance(NotebookInstanceName=notebook_instance_name)
security_group_id = notebook_instance['SecurityGroups'][0]
print(security_group_id)

sg-09d6dc1d0550338ba


### Create Secret in Secrets Manager

AWS Secrets Manager is a service that enables you to easily rotate, manage, and retrieve database credentials, API keys, and other secrets throughout their lifecycle. Using Secrets Manager, you can secure and manage secrets used to access resources in the AWS Cloud, on third-party services, and on-premises.

*note that `MasterUserPassword` must contain at least 1 upper case letter and at least 1 decimal digit.

In [26]:
secretsmanager = boto3.client('secretsmanager')

try:
    response = secretsmanager.create_secret(
        Name='tabular_redshift_login',
        Description='Boston House data New Cluster Redshift Login',
        SecretString='[{"username":"awsuser"},{"password":"Bostonhouse1"}]',
        Tags=[
            {
                'Key': 'name',
                'Value': 'tabular_redshift_login'
            },
        ]
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceExistsException':
        print("Secret already exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

Secret already exists. This is ok.


In [27]:
# And retrieving the secret again
secretsmanager = boto3.client('secretsmanager')
import json

secret = secretsmanager.get_secret_value(SecretId='tabular_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']

In [28]:
# Set up parameters          
# Redshift configuration parameters
redshift_cluster_identifier = 'redshiftdemo'
database_name = 'bostonhouse'
cluster_type = 'multi-node'

node_type = 'dc2.large'
number_nodes = '2' 

When creating a new cluster, you want ot make sure that the Redshift VPC is the same one you create your notebook in. Your VPC should has the following two VPC attributes set to yes: **DNS resolution** and **DNS hostnames**. You can either specify a **security group* or specify a created **cluster subnet group name** (which you will create from the Redshift console).

If you are not using default VPC and using **security group** returns VPC error, you can try create a subnet group in Redshift Console, by choose **Configurations** -> **subnet groups** -> **create cluster subnet group**, then specify the **VPC** and **subnet** you want to choose and you created this notebook in. Specify the `ClusterSubnetGroupName` in the following command with the subnet group you created.

In [31]:
response = redshift.create_cluster(
        DBName=database_name,
        ClusterIdentifier=redshift_cluster_identifier,
        ClusterType=cluster_type,
        NodeType=node_type,
        NumberOfNodes=int(number_nodes),       
        MasterUsername=master_user_name,
        MasterUserPassword=master_user_pw,
       ClusterSubnetGroupName='cluster-subnet-group-1', #you can either specify an existing subnet group (change this to your Subnet Group name), or specify your security group below
        IamRoles=[iam_role_redshift_arn],
        VpcSecurityGroupIds=[security_group_id],
        Port=5439,
      PubliclyAccessible=False
)

print(response)

{'Cluster': {'ClusterIdentifier': 'redshiftdemo', 'NodeType': 'dc2.large', 'ClusterStatus': 'creating', 'ClusterAvailabilityStatus': 'Modifying', 'MasterUsername': 'awsuser', 'DBName': 'bostonhouse', 'AutomatedSnapshotRetentionPeriod': 1, 'ManualSnapshotRetentionPeriod': -1, 'ClusterSecurityGroups': [], 'VpcSecurityGroups': [{'VpcSecurityGroupId': 'sg-09d6dc1d0550338ba', 'Status': 'active'}], 'ClusterParameterGroups': [{'ParameterGroupName': 'default.redshift-1.0', 'ParameterApplyStatus': 'in-sync'}], 'ClusterSubnetGroupName': 'cluster-subnet-group-1', 'VpcId': 'vpc-037a0a282233de8de', 'PreferredMaintenanceWindow': 'wed:05:00-wed:05:30', 'PendingModifiedValues': {'MasterUserPassword': '****'}, 'ClusterVersion': '1.0', 'AllowVersionUpgrade': True, 'NumberOfNodes': 2, 'PubliclyAccessible': False, 'Encrypted': False, 'Tags': [], 'EnhancedVpcRouting': False, 'IamRoles': [{'IamRoleArn': 'arn:aws:iam::060356833389:role/Tabular_Redshift', 'ApplyStatus': 'adding'}], 'MaintenanceTrackName': 'cu

Wait until the status of your redshift cluster become **available**.

In [33]:
#check cluster status
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print('Your Redshift Cluster Status is: ' + cluster_status)

Your Redshift Cluster Status is: available


# Existing Redshift Cluster
### Prerequisites
Your existing Redshift cluster have to be in the **same VPC** as your notebook instance.

Also, note that this Notebook instance needs to resolve to a private IP when connecting to the Redshift instance. There are two ways to resolve the Redshift DNS name to a private IP:

The Redshift cluster is not publicly accessible so by default it will resolve to private IP.
The Redshift cluster is publicly accessible and has an EIP associated with it but when accessed from within a VPC, it should resolve to private IP of the Redshift cluster. This is possible by setting following two VPC attributes to yes: **DNS resolution** and **DNS hostnames**. For instructions on setting that up, see Redshift public docs on [Managing Clusters in an Amazon Virtual Private Cloud (VPC)](https://docs.aws.amazon.com/redshift/latest/mgmt/managing-clusters-vpc.html).

We will use [sqlalchemy](https://pypi.org/project/SQLAlchemy/) to connect to the redshift database engine.

In [34]:
!{sys.executable} -m pip install -qU SQLAlchemy==1.3.13

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [35]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

#### Retrive Redshift credentials from Secret Manager

In [36]:
import json
secretsmanager = boto3.client('secretsmanager')
secret = secretsmanager.get_secret_value(SecretId='tabular_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']

#### Set up parameters for connection: replace with your own parameters

In [37]:
redshift_cluster_identifier = 'redshiftdemo'

database_name_redshift = 'bostonhouse'
database_name_athena = 'tabularbh'

redshift_port = '5439'

schema_redshift = 'redshift'
schema_spectrum = 'spectrum'

table_name_csv = 'boston_house_athena'

#### Check cluster status to see if it is available

In [38]:
#check cluster status
response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

available


In [39]:
# Get Redshift Endpoint Address & IAM Role
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print('Redshift endpoint: {}'.format(redshift_endpoint_address))
print('IAM Role: {}'.format(iam_role))

Redshift endpoint: redshiftdemo.ce68y8q9neek.us-east-2.redshift.amazonaws.com
IAM Role: arn:aws:iam::060356833389:role/Tabular_Redshift


#### Create Engine
https://docs.sqlalchemy.org/en/13/core/engines.html

In [40]:
# Connect to Redshift Database Engine
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(master_user_name, master_user_pw, redshift_endpoint_address, redshift_port, database_name_redshift))

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


#### Create Session: we will use this session to run SQL commands

In [41]:
# config session
session = sessionmaker()
session.configure(bind=engine)
s = session()

## Method 1: Access Data without Moving it to Redshift: Amazon Redshift Spectrum
[Redshift Spectrum](https://docs.aws.amazon.com/redshift/latest/dg/c-getting-started-using-spectrum.html) is used to query data directly from files on Amazon S3.You will need to create external tables in an external schema. The external schema references a database in the external data catalog and provides the IAM role ARN that authorizes your cluster to access Amazon S3 on your behalf.
#### Get table and schema information from the Glue Catalog: getting meta data from data catalog and connecting to the Athena database

In [42]:
statement = """
rollback;
create external schema if not exists {} from data catalog 
    database '{}' 
    iam_role '{}'
    create external database if not exists
""".format(schema_spectrum, database_name_athena, iam_role)

s.execute(statement)
s.commit()

#### Run a sample query through Redshift Spectrum

In [44]:
import pandas as pd
statement = """
select *
    from {}.{} limit 10
""".format(schema_spectrum, table_name_csv)

df = pd.read_sql_query(statement, engine)
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## Method 2: Loading Data into Redshift from Athena
To load data into Redshift, you need to either use `COPY` command or `INSERT INTO` command to move data into a table from data files. Copied files may reside in an S3 bucket, an EMR cluster, or on a remote host accessed.
#### Create Schema in Redshift

In [45]:
#create schema
statement = """create schema if not exists {}""".format(schema_redshift)

s = session()
s.execute(statement)
s.commit()

#### Create Redshift Table

In [46]:
table_name_redshift = table_name_csv+'_'+'redshift_insert'
statement = """
rollback;
create table if not exists redshift.{}(
        CRIM float,
        ZN float,
        INDUS float,
        CHAS float,
        NOX float,
        RM float,
        AGE float,
        DIS float, 
        RAD float, 
        TAX float,
        PTRATIO float, 
        B float, 
        LSTAT float,
        target float)""".format(table_name_redshift)

s.execute(statement)
s.commit()

#### `Insert into` data into the table we created
https://docs.aws.amazon.com/redshift/latest/dg/c_Examples_of_INSERT_30.html

In [47]:
table_name_redshift = table_name_csv+'_'+'redshift_insert'

statement = """
    insert into redshift.{}
        select * from {}.{}             
    """.format(table_name_redshift, schema_spectrum, table_name_csv)
s.execute(statement)
s.commit()        

#### Query data in Redshift

In [48]:
statement = """
     select * from redshift.{} limit 10
""".format(table_name_redshift)
df = pd.read_sql_query(statement, engine)
df.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
1,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
2,0.22489,12.5,7.87,0.0,0.524,6.377,94.3,6.3467,5.0,311.0,15.2,392.52,20.45,15.0
3,0.63796,0.0,8.14,0.0,0.538,6.096,84.5,4.4619,4.0,307.0,21.0,380.02,10.26,18.2
4,0.80271,0.0,8.14,0.0,0.538,5.456,36.6,3.7965,4.0,307.0,21.0,288.99,11.69,20.2


## Method 3: Copy data directly from S3
You can also `Copy` Data into a new table.
https://docs.aws.amazon.com/redshift/latest/dg/tutorial-loading-run-copy.html

#### Create a new Schema in Redshift

In [49]:
#create a new sample table
table_name_redshift = table_name_csv+'_'+'redshift_copy'
statement = """
rollback;
create table if not exists redshift.{}(
        CRIM float,
        ZN float,
        INDUS float,
        CHAS float,
        NOX float,
        RM float,
        AGE float,
        DIS float, 
        RAD float, 
        TAX float,
        PTRATIO float, 
        B float, 
        LSTAT float,
        target float)""".format(table_name_redshift)

s.execute(statement)
s.commit()

#### Copy data into Redshift table
Redshift assumes your data comes in pipe delimited, so if you are reading in csv or txt, be sure to specify the `delimiter`. To load data that is in `CSV` format, add `csv` to your `COPY` command. Also since we are reading directly from S3, if your data has header, remember to add `ignoreheader` to your command.

In [50]:
table_name_redshift = table_name_csv+'_'+'redshift_copy'
data_s3_path = 's3://sagemaker-us-east-2-060356833389/data/tabular/boston_house/boston_house.csv'
statement = """
rollback;
copy redshift.{}  
  from '{}'
  iam_role '{}'
  csv
  ignoreheader 1
    """.format(table_name_redshift, data_s3_path, iam_role)
s.execute(statement)
s.commit() 

In [51]:
statement = """
     select * from redshift.{} limit 10
""".format(table_name_redshift)
df_copy = pd.read_sql_query(statement, engine)
df_copy.head(5)

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### Error Handling
https://docs.aws.amazon.com/redshift/latest/dg/r_STL_LOAD_ERRORS.html
Sometimes you might see an error stating" Load into table 'part' failed.  Check 'stl_load_errors' system table for details.", and below is a helpful function helping you checking where the copying process went wrong.

In [59]:
import pandas as pd
statement = """
select query, substring(filename,22,25) as filename,line_number as line, 
substring(colname,0,12) as column, type, position as pos, substring(raw_line,0,30) as line_text,
substring(raw_field_value,0,15) as field_text, 
substring(err_reason,0,45) as reason
from stl_load_errors 
order by query desc
limit 10"""
error = pd.read_sql_query(statement, engine)
error.head(5)

,query,filename,line,column,type,pos,line_text,field_text,reason
0,2173,t-2-060356833389/data/tab,2,chas,int4,18,"0.00632,18.0,2.31,0.0,0.538,6",0.0,"Invalid digit, Value '.', Pos 1, Type: Integ"
1,2147,t-2-060356833389/data/tab,1,crim,float8,0,"CRIM,ZN,INDUS,CHAS,NOX,RM,AGE",CRIM,"Invalid digit, Value 'C', Pos 0, Type: Doubl"
2,2083,t-2-060356833389/data/tab,1,crim,float8,0,"CRIM,ZN,INDUS,CHAS,NOX,RM,AGE",CRIM,"Invalid digit, Value 'C', Pos 0, Type: Doubl"
3,2063,t-2-060356833389/data/tab,1,crim,float8,0,"CRIM,ZN,INDUS,CHAS,NOX,RM,AGE",CRIM,"Invalid digit, Value 'C', Pos 0, Type: Doubl"
4,2049,t-2-060356833389/data/tab,1,crim,float8,0,"CRIM,ZN,INDUS,CHAS,NOX,RM,AGE",CRIM,"Invalid digit, Value 'C', Pos 0, Type: Doubl"


## Method 4: AWS Data Wrangler

https://github.com/awslabs/aws-data-wrangler/blob/master/tutorials/008%20-%20Redshift%20-%20Copy%20%26%20Unload.ipynb

In [52]:
!{sys.executable} -m pip install -qU awswrangler==1.2.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


### AWS Data Wrangler Get Engine Function
Run this command within a private subnet. You can find your host address by going to the Redshift Console, then choose **Clusters** -> **Property** -> **Connection details** -> **View all connection details** -> **Node IP address** -> **Private IP address**.
https://aws-data-wrangler.readthedocs.io/en/latest/stubs/awswrangler.db.get_engine.html#awswrangler.db.get_engine

In [53]:
import awswrangler as wr
import pandas as pd
from datetime import date

engine = wr.db.get_engine(
    db_type="postgresql",
    host= '10.0.14.121', #Private IP address of your Redshift Cluster
    port=redshift_port,
    database=database_name_redshift,
    user = master_user_name,
    password=master_user_pw
)

In [54]:
df = wr.db.read_sql_query("SELECT * FROM redshift.{}".format(table_name_redshift), con=engine)

In [55]:
df.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2
